In [1]:
import pandas as pd
import numpy as np

In [30]:
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

In [3]:
from sklearn.impute import SimpleImputer

In [4]:
from functions import *

In [5]:
X = pd.read_csv('X_ccrf.csv')

cols_2_impute =[]
print(X.shape[0])
#print("-------------------------------------------------------")
for column in X.columns:
    if '?' in X[column].unique():
        #print('{} : {}'.format(column,df[column].value_counts()['?']))
        cols_2_impute.append(column)
print(len(cols_2_impute))

858
24


/Users/tom/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


In [6]:
new_X = X.replace('?',np.nan)
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_X = pd.DataFrame(imp.fit_transform(new_X),columns=new_X.columns)

In [7]:
y = pd.read_csv('y_ccrf.csv')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(imp_X, y, test_size=0.30, random_state=101)

In [9]:
type(y_train)

pandas.core.frame.DataFrame

In [10]:
df_concat = pd.concat([X_train,y_train],axis=1)
df_concat.head()

,Hormonal Contraceptives (years),STDs:HIV,Smokes (years),First sexual intercourse,Smokes (packs/year),IUD (years),IUD,Dx:HPV,STDs:molluscum contagiosum,STDs: Number of diagnosis,...,Dx:CIN,STDs:HPV,Smokes,Number of sexual partners,STDs:genital herpes,STDs:vaginal condylomatosis,Hinselmann,Schiller,Citology,Biopsy
29,0.25,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.000000,0.0,0.0,0,0,0,0
517,5.00,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,0.0,0.0,0,0,0,0
464,0.08,0.0,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,0.0,0.0,0,1,0,0
752,0.50,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.527644,0.0,0.0,0,0,0,0
568,0.00,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.000000,0.0,0.0,0,0,0,0


In [11]:
#validation, training = cross_validation(df_test,cv=5)
targets = ['Hinselmann','Schiller','Citology','Biopsy']
clf_train = tree.DecisionTreeClassifier()
results = cross_validation(df_concat,targets,clf_train,cv=10)

In [12]:
print(results)

[0.75, 0.82, 0.87, 0.68, 0.73, 0.82, 0.78, 0.75, 0.7, 0.7]


In [13]:
average_cv_accuracy(results)

0.76

In [15]:
targets = ['Hinselmann','Schiller','Citology','Biopsy']
clf_train = tree.DecisionTreeClassifier()
results = cross_validation(df_concat,targets,clf_train,cv=5)
print(results)
print(average_cv_accuracy(results))

[0.85, 0.82, 0.76, 0.81, 0.78]
0.8


In [16]:
targets = ['Hinselmann','Schiller','Citology','Biopsy']
clf_train = tree.DecisionTreeClassifier(max_depth=4,max_features='log2',min_samples_split=4,min_impurity_decrease=0.5)
results = cross_validation(df_concat,targets,clf_train,cv=5)
print(results)
print(average_cv_accuracy(results))

[0.87, 0.88, 0.88, 0.89, 0.9]
0.88


In [19]:
y_train_labels =[]
for idx in y_train.index:
    row =[str(y_train.loc[idx][0]),str(y_train.loc[idx][1]),str(y_train.loc[idx][2]),str(y_train.loc[idx][3])]
    y_train_labels.append(row)

In [27]:
y_test_labels =[]
for idx in y_test.index:
    row =[str(float(y_test.loc[idx][0])),str(float(y_test.loc[idx][1])),
          str(float(y_test.loc[idx][2])),str(float(y_test.loc[idx][3]))]
    y_test_labels.append(row)

In [21]:
clf_main = tree.DecisionTreeClassifier(max_depth=4,max_features='log2',min_samples_split=4,min_impurity_decrease=0.5)

In [22]:
clf_main.fit(X_train,y_train_labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features='log2', max_leaf_nodes=None,
            min_impurity_decrease=0.5, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [23]:
preds = clf_main.predict(X_test)

In [24]:
new_preds = transform_preds(preds)

In [28]:
y_test_labels[0]

['0.0', '0.0', '0.0', '0.0']

In [29]:
print(multilabel_accuracy(y_test_labels,new_preds))

0.88
